# Simiosis gia Grid
- o skopos tou na dimiourgisw ena grid einai gia na miosw kata poli to sinolo ton simion sto disance join
- auto epitinxanete ftiaxnontas ena MBB pernontas ta min ton max(x,y) kai apo ta 2 data sets kathos kai ta max apo ta min(x,y) me tin logiki oti osa simia einai ektos tou min max x enos apo ta 2 dataset den ikanopoioun tin sinthiki theta.
- stin sinexia tha prepei na kanw spatial join to grid kai me ta 2 datasets gia na vrw se poio grid paei kathe simio
- meta tha aporipsw osa grids den exoun kanena

### Xrisima stixia na periexei ena grid
- gridId
- polygon
- TODO Check id x an y needed for partitioning reasons

### Idea gia grid filtering
- Na kanw spatial join mono sto dataset me ta ligotera simia prokimenou na filtrarw ta grids pou den exeoun kanena simio apo to Dataset auto
- Me auton ton tropo epitinxanw na kanw mikrotero spatial join sto dataset me tis perisoteres eggrafes
- auto vasizete sto

# Simiosis gia partitioning
- prepei na xorisw ta data mou se ola ta mixanimata prin ksekinisw to spatial join
- episis prepei na xorisw kai ta grids omiomorfa se ola ta mixanimata
- enas tropos gia na ta xorisw tha mporouse na einai to X. Diladi na vrw ta max kai min X kai paota 2 datasets gia na vrw to rangekai na kanw range partition os pros x gia ta points
- gia ta grids apo tin all tha eprepe na kanw partition os pros to max X tous oste na simenei oti oti grid einai sto partition exei points mono sto partition auto.
- me auton ton tropo epitinxanoume na exoume ta points kai ta grids tous mirasmena ana mixanima kai na empodisoume tis metakinisis sto diktio

### Pws to kanw auto?
- prepei na dw pws douleui to pratitioning
- pos simperiferete to join sto partitioning
- prepei na dw an ipostirizei kati tetoio i vivliothiki

## Imports

In [2]:
# Main  Imports
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython.core.display import display
import time

# Local App Imports
from utils import spatialUtils as utils

# Spark Imports
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, LongType


# Sedona Imports
from sedona.utils import KryoSerializer, SedonaKryoRegistrator
from sedona.register import SedonaRegistrator
from sedona.sql.types import GeometryType

# Python shapely imports
import shapely.geometry as sg
from shapely.geometry import Polygon
from descartes import PolygonPatch
import geog
import shapely.speedups
shapely.speedups.enable() # this makes some spatial queries run faster

ModuleNotFoundError: No module named 'pandas._libs.join'

## Load Dataset

In [ ]:
# full program timer
total_time = time.time()

test_sample_A_path = "../testData/ais_one_hour.csv"
test_sample_B_path = "../testData/ais_one_hour2.csv"

# test_sample_path = "../testData/ais_one_month.csv"


# create sedona spark session
spark = SparkSession. \
    builder. \
    appName('appName'). \
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.0-incubating,'
           'org.datasyslab:geotools-wrapper:geotools-24.0'). \
    getOrCreate()

SedonaRegistrator.registerAll(spark)

# Load CSV File
df_a = spark.read.csv(header="True", inferSchema="True", path=test_sample_A_path)
df_a.show(10)

df_b = spark.read.csv(header="True", inferSchema="True", path=test_sample_B_path)
df_b.show(10)

## Filter Dataset for Test Reasons
- arxika tha periorisoume to dataset stin perioxi x[-6.5,-5] kai y[47.5,49]
- epilegw auto gt kserw oti exei arketa pings

df_a = df_a.filter( (-6 <= df_a.X) & (df_a.X <= -5) & (47.5 <= df_a.Y) & (df_a.Y <= 49))\
    .cache()
print("Total filtered_df_a count {}".format(df_a.count()))
df_a.show(10)

df_b = df_b.filter( (-6.5 <= df_b.X) & (df_b.X <= -5) & (47.5 <= df_b.Y) & (df_b.Y <= 49))\
    .cache()
print("Total filtered_df_b count {}".format(df_b.count()))
df_b.show(10)

## Grid Dataset
Lets create a test grid
- We have to find the MBB from the dataset with less records
- Then we have to create a grid for this dataset (**sos** each grid side has to be equal to theta)
- Then calculate expanded grids for each grid

### Times:
first run --- 4.288848161697388 seconds ---

In [ ]:
def calculate_MBB_from_sdf(sdf):
    # find min/max X,Y
    mbb_row = sdf.agg(
        F.max(sdf.X),
        F.min(sdf.X),
        F.max(sdf.Y),
        F.min(sdf.Y),
    ).collect()[0]
    print("print mbb: {}".format(mbb_row))
    #calculate grid for dataset
    return utils.getPolyGrid(mbb_row["min(X)"],mbb_row["min(Y)"],mbb_row["max(X)"],mbb_row["max(Y)"], theta=10)



In [ ]:
cell_time = time.time()

#calculate grid for dataset
grid = calculate_MBB_from_sdf(df_a) if df_a.count() <= df_b.count() else calculate_MBB_from_sdf(df_b)
print("grid len: {}".format(len(grid)))

print("--- %s seconds ---" % (time.time() - cell_time))


## Create Grid RDD or DF

In [ ]:
grid_df = spark.createDataFrame(grid)
grid_df.printSchema()
print(grid_df.schema)
grid_df.show(10)

## Link Points tou Grids
- **Approach 1:** Test databricks code for spatial join from link
- **Approach 2:** perform spatial join in order to link points with grids
- **Approach 3:** perform spatial join without using intersection. use where statement on join and check if point is within grid box

### Results
It seems that approach 3 is 3 times faster than approach 2 because we dont need to check for intersection using apache
sedona because we use squares/grids as polygons and in this case we can easily check if point intersect with the square
grid by chacking max and min X,Y respectively. Despite that we dont need to use UDF function in order to conver X,Y coords into sapely point

### Approach 3

In [ ]:
# timing procedure
start_time = time.time()

df_a.createOrReplaceTempView("points")
grid_df.createOrReplaceTempView("grids")

spatial_join_result = spark.sql(
    """
        SELECT *
        FROM points AS P join grids AS G
        WHERE P.X >= G.grid_minX and P.X <= G.grid_maxX
         and P.Y >= G.grid_minY and P.Y <= G.grid_maxY
    """
).cache() # cache results to avoid perform join again



print("--- %s seconds ---" % (time.time() - start_time))

# print some stats
spatial_join_result.show()
print(spatial_join_result.count())

## Spatial join Approach 2 stats
- find count
- find pings per grid
- filter grids with 0 points

### SOS
Otan kanoume chaining functions mporoume na exoume access se colum apo to proigoumeno function kanontas F.col("coll_name")

In [ ]:
print("spatial join count: {}".format(spatial_join_result.count()))

# find pings per grid
filtered_grid = spatial_join_result.groupby(["grid_id", "grid_maxX", "grid_maxY", "grid_minX", "grid_minY", "exp_grid_maxX", "exp_grid_maxY", "exp_grid_minX", "exp_grid_minY"])\
    .count()\
    .filter(F.col("count") > 0)\
    .cache()

filtered_grid.show(10)

print("filtered spatial join count: {}".format(filtered_grid.count()))


## Perform Spatial Join With Point_set B And expanded gids

By performing this spatial join we can filter points not in expanded grids (if exist)
and keep only those points who have a chance to satisfy condition theta

for This spatial join we choose approach 3 from previous spatial join (datasetA with grids)

In [ ]:
# timing procedure
start_time = time.time()

df_b.createOrReplaceTempView("points_B")
filtered_grid.createOrReplaceTempView("grids")

spatial_join_b_result_new = spark.sql(
    """
        SELECT P._id, P.X, P.Y, G.grid_id as exp_grid_id, G.grid_minX, G.grid_maxX, G.grid_minY, G.grid_maxY
        FROM points_B AS P, grids AS G
        WHERE P.X >= G.exp_grid_minX and P.X <= G.exp_grid_maxX
         and P.Y >= G.exp_grid_minY and P.Y <= G.exp_grid_maxY
    """
).withColumn("grid_id", F.when((F.col("X") >= F.col("grid_minX")) &
                                  (F.col("X") <= F.col("grid_maxX")) &
                                  (F.col("Y") >= F.col("grid_minY")) &
                                  (F.col("Y") <= F.col("grid_maxY")),F.col("exp_grid_id"))
                           .otherwise(None))\
    .cache() # cache results to avoid perform join again

spatial_join_b_result_new.show()
print("--- %s seconds ---" % (time.time() - start_time))

def createPoints(longitude, latitude):
  return sg.Point(longitude, latitude)

createPoints_udf = F.udf(createPoints,GeometryType())

## Get filtered point set B


In [ ]:
print("spatial join count: {}".format(spatial_join_b_result_new.count()))

# find pings per grid
geom_b_df = spatial_join_b_result_new.groupby(["_id", "X", "Y"])\
    .count()\
    .filter(F.col("count") > 0)\
    .cache()

geom_b_df.show(10)

print("filtered spatial join count: {}".format(geom_b_df.count()))

## Find matching points/pairs

As a first filter of matching point we use the fact that each pair point in the same
grid between 2 datasets satisfied the theta condition so:
- We perform a filter on point set B in order to keep only those points that has been
assigned in a grid
- then we perform a simple join with points from point set A based on grid id to find all
valid pairs given by grids
- then write the result into a file (override result if file exits)

### SOS gia tin ilopoiisi
- to filtrarisma tha ginei kratontas osa point exoun idio grid kai expanded grid id

In [ ]:
# timing procedure
start_time = time.time()

filtered_a_results = spatial_join_result\
    .select(F.col("_id"), F.col("grid_id"))\
    .withColumnRenamed("_id","a_id")

filtered_b_results = spatial_join_b_result_new\
    .filter(F.col("grid_id") == F.col("exp_grid_id"))\
    .select(F.col("_id"), F.col("grid_id"))\
    .withColumnRenamed("_id","b_id")

ta = filtered_a_results.alias('ta')
tb = filtered_b_results.alias('tb')

first_result = ta.join(tb, ta.grid_id == tb.grid_id)\
    .select(F.col("a_id"), F.col("b_id"))

# write file
print(first_result.count())#.write.save("dist_join_result.csv", format="csv", mode="overwrite")

print("--- %s seconds ---" % (time.time() - start_time))

## Find matching points using expanded grids
Now we have to check for matching pairs between points from point set A that are in
grid and points from pointest B that ar in the same expanded grid and not in the grid

- declare a udf for haversian distance
- first get all points with null grid from joined pointset B
-

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

udfsomefunc = F.udf(haversine, DoubleType())

In [ ]:
# timing procedure
start_time = time.time()

# register udf func
haversine_dist_udf = F.udf(utils.calculatePointsDistance, DoubleType())

filtered_a_in_exp_grid = spatial_join_result\
    .withColumnRenamed("_id","a_id")\
    .withColumnRenamed("X","a_X")\
    .withColumnRenamed("Y","a_Y")\
    .withColumnRenamed("grid_id","a_grid_id")\
    .select(F.col("a_id"),F.col("a_X"),F.col("a_Y"),F.col("a_grid_id"))\

filtered_b_in_exp_grid = spatial_join_b_result_new.where(F.col("grid_id").isNull())\
    .withColumnRenamed("_id","b_id")\
    .withColumnRenamed("X","b_X")\
    .withColumnRenamed("Y","b_Y")\
    .withColumnRenamed("exp_grid_id","b_exp_grid_id")\
    .select(F.col("b_id"),F.col("b_X"),F.col("b_Y"),F.col("b_exp_grid_id"))\


ta = filtered_a_in_exp_grid.alias('ta')
tb = filtered_b_in_exp_grid.alias('tb')

final_result = ta.join(tb, ta.a_grid_id == tb.b_exp_grid_id)

final_result = final_result.withColumn("haversine_dist", udfsomefunc(F.col("a_X"),F.col("a_Y"),F.col("b_X"),F.col("b_Y")))\
    .filter(F.col("haversine_dist") <= 10)
#\
    # .where(createPoints_udf(F.col("a_X"),F.col("a_Y"),F.col("b_X"),F.col("b_Y")) >= 10)#\
    # .select(F.col("a_id"), F.col("b_id"))

# final_result.write.save("dist_join_result.csv", format="csv", mode="append")

final_result.show(10)
print(final_result.count())

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print("--- TOTAL EXECUTION TIME ---\n--- %s seconds ---" % (time.time() - total_time))

